In [1]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo", 
    temperature=0
)

In [2]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [3]:
from langchain_core.documents import Document

# Define a list of documents with content and metadata
documents = [
    Document(
        page_content="The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.",
        metadata={"source": "cricket news"},
    ),
    Document(
        page_content="The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.Both ties promise thrilling encounters, featuring some of the best talents in world football.",
        metadata={"source": "football news"},
    ),
    Document(
        page_content="As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives.",
        metadata={"source": "election news"},
    ),
    Document(
        page_content="The AI revolution continues to transform industries and reshape the global economy.Significant advancements in artificial intelligence have led to breakthroughs in healthcare, with AI-driven diagnostics improving patient outcomes and reducing costs.Autonomous systems are becoming increasingly prevalent in logistics and transportation, enhancing efficiency and safety.",
        metadata={"source": "ai revolution news"},
    ),
]

In [4]:
from langchain_chroma import Chroma

vectorStore = Chroma.from_documents(documents=documents, embedding=embedding_model)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [5]:
results = vectorStore.similarity_search("test match")

for result in results:
    print("-"*100)
    print(result.page_content)
    print(result.metadata)
    

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


----------------------------------------------------------------------------------------------------
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
{'source': 'cricket news'}
----------------------------------------------------------------------------------------------------
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, whil

In [20]:
query_embedding = embedding_model.embed_query("election ")
print(query_embedding[:10])

[0.03525533154606819, -0.02910839021205902, 0.033463623374700546, -0.00972347054630518, 0.014443936757743359, -0.02717885747551918, -0.015491397120058537, 0.046143416315317154, -0.07403895258903503, 0.010660672560334206]


In [21]:
results = vectorStore.similarity_search_by_vector(query_embedding)

for result in results:
    print("-"*100)
    print(result.page_content)
    print(result.metadata)

----------------------------------------------------------------------------------------------------
As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives.
{'source': 'election news'}
----------------------------------------------------------------------------------------------------
The world of football is buzzing with excitement as major tournaments and league matches continue to captivate fans globally.In the UEFA Champions League, the semi-final matchups have been set, with defending champions Real Madrid set to face Manchester City, while Bayern Munich will take on Paris Saint-Germain.Both ties 

In [30]:
retriever = vectorStore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

batch_results = retriever.batch(["test match", "election"])

for result in batch_results:
    print("-"*100)
    for r in result:
        print(r.page_content)
        print(r.metadata)

----------------------------------------------------------------------------------------------------
The T20 World Cup 2024 is in full swing, bringing excitement and drama to cricket fans worldwide.India's team, captained by Rohit Sharma, is preparing for a crucial match against Ireland, with standout player Jasprit Bumrah expected to play a pivotal role in their campaign.The tournament has already seen controversy, particularly concerning the pitch conditions at Nassau County International Cricket Stadium in New York, which came under fire after a low-scoring game between Sri Lanka and South Africa.
{'source': 'cricket news'}
----------------------------------------------------------------------------------------------------
As election season heats up, the latest developments reveal a highly competitive atmosphere across several key races.The presidential election has seen intense campaigning from all major candidates, with recent polls indicating a tight race.Incumbent President Jan

In [94]:
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_messages([
    ("human", message)
])

In [95]:
chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

question = "Current status of the election?"
response = chain.invoke(question)

history.extend([
    ("human", question),
    ("ai", response.content)
])


response.content

'The current status of the election is that it is highly competitive, with intense campaigning from all major candidates and recent polls indicating a tight race. Incumbent President Jane Doe is seeking re-election on a platform of economic stability and healthcare reform, while her main rival, Senator John Smith, focuses on education and climate change initiatives.'

In [93]:
history

[('human', 'Current status of the election?'),
 ('ai',
  'The current status of the election is highly competitive with intense campaigning from major candidates, indicating a tight race between incumbent President Jane Doe and Senator John Smith.'),
 ('human', 'Current status of the election?'),
 ('ai',
  'The current status of the election is highly competitive with intense campaigning from major candidates, indicating a tight race between incumbent President Jane Doe and Senator John Smith.')]